In [8]:
import io
from IPython.nbformat import current
def execute_notebook(nbfile):
    with io.open(nbfile) as f:
        nb = current.read(f, 'json')
    ip = get_ipython()
    for cell in nb.worksheets[0].cells:
        if cell.cell_type != 'code':
            continue
        ip.run_cell(cell.input)
execute_notebook("/cellar/users/ramarty/Projects/hla_ii/bin/imports.ipynb")
execute_notebook("/cellar/users/ramarty/Projects/hla_ii/bin/gathering_affinities.ipynb")

Populating the interactive namespace from numpy and matplotlib


/cellar/users/ramarty/anaconda/lib/python2.7/site-packages/IPython/nbformat/current.py:19: UserWarning: IPython.nbformat.current is deprecated.

- use IPython.nbformat for read/write/validate public API
- use IPython.nbformat.vX directly to composing notebooks of a particular version

  """)


In [9]:
donor_dictionary = {}
for f in os.listdir('/cellar/users/ramarty/Data/hla_ii/validation/ciudad/raw/'):
    donor = f.split('_')[0]
    alleles = [x[:4]+'_'+x[4:] for x in f.split('.')[0].split('_')[1:]]
    donor_dictionary[donor] = alleles

In [10]:
donor_dictionary

{'DonorA': ['DRB1_0301', 'DRB1_1101'],
 'DonorB': ['DRB1_0401', 'DRB1_1301'],
 'DonorC': ['DRB1_0401', 'DRB1_1301'],
 'DonorD': ['DRB1_0101', 'DRB1_0701'],
 'DonorE': ['DRB1_0101', 'DRB1_1101'],
 'DonorF': ['DRB1_0901', 'DRB1_1001'],
 'DonorG': ['DRB1_0701', 'DRB1_1501', 'DRB5_0101']}

In [11]:
# input allele and donors?? 
alleles, donors = [], []
for donor in donor_dictionary.keys():
    alleles.extend(donor_dictionary[donor])
    donors.extend([donor for x in donor_dictionary[donor]])
lengths = [str(x) for x in range(13, 26)]
print len(alleles), len(donors), len(lengths)

15 15 13


In [13]:
all_alleles = alleles*len(lengths)
all_donors = donors*len(lengths)
all_lengths = []
for length in lengths:
    all_lengths.extend([length]*15)
print len(all_alleles), len(all_donors), len(all_lengths)

195 195 195


In [24]:
def create_cluster_script(alleles, donors, lengths):

    new_script_file = '/cellar/users/ramarty/Projects/hla_ii/data_gathering/affinities/validation/netmhcpanII.ciudad.sh'
    
    with open(new_script_file, 'w') as out_file:
        out_file.write("#! /bin/csh\n")
        out_file.write("#$ -V\n")
        out_file.write("#$ -S /bin/csh\n")
        out_file.write("#$ -o /cellar/users/ramarty/Data/hla_ii/sge-system_files\n")
        out_file.write("#$ -e /cellar/users/ramarty/Data/hla_ii/sge-system_files\n")
        out_file.write("#$ -cwd\n")
        out_file.write("#$ -t 1-{0}\n".format(len(alleles)))
        #out_file.write("#$ -t 1-2\n".format(len(samples)))
        out_file.write("#$ -l h_vmem=1G\n")
        out_file.write("#$ -tc 200\n")
        out_file.write("#$ -l long")
        out_file.write("\n")

        out_file.write("set alleles=({0})\n".format(" ".join(alleles)))
        out_file.write("set donors=({0})\n".format(" ".join(donors)))
        out_file.write("set lengths=({0})\n".format(" ".join(lengths)))
        out_file.write("\n")

        out_file.write("set allele=$alleles[$SGE_TASK_ID]\n")
        out_file.write("set donor=$donors[$SGE_TASK_ID]\n")
        out_file.write("set length=$lengths[$SGE_TASK_ID]\n")
        out_file.write("\n")

        out_file.write("date\n")
        out_file.write("hostname\n")
        out_file.write("\n")
  
        out_file.write("echo starting netMHCIIpan\n")
        out_file.write("/cellar/users/ramarty/programs/netMHCIIpan-3.1/netMHCIIpan -f /cellar/users/ramarty/Data/hla_ii/validation/ciudad/fasta_files/$donor.$length.fa -a $allele -length $length -xls -xlsfile /cellar/users/ramarty/Data/hla_ii/validation/ciudad/affinities/$donor.$allele.$length.csv > /cellar/users/ramarty/Data/hla_ii/sge-system_files/$donor.$allele.$length.txt\n")
        out_file.write("/cellar/users/ramarty/programs/netMHCIIpan-3.1/netMHCIIpan -f /cellar/users/ramarty/Data/hla_ii/validation/ciudad/fasta_files/$donor.random.$length.fa -a $allele -length $length -xls -xlsfile /cellar/users/ramarty/Data/hla_ii/validation/ciudad/affinities/$donor.$allele.random.$length.csv > /cellar/users/ramarty/Data/hla_ii/sge-system_files/$donor.$allele.random.$length.txt\n")
        out_file.write("\n")
                       
        out_file.write("date\n")

In [25]:
create_cluster_script(all_alleles, all_donors, all_lengths)

In [7]:
! /cellar/users/ramarty/programs/netMHCIIpan-3.1/netMHCIIpan -f /cellar/users/ramarty/Data/hla_ii/presentation/fasta_files/test.fa -a DRB1_0301 -xls -xlsfile /cellar/users/ramarty/Data/hla_ii/presentation/affinities/random/test.csv -length {lengths}

# NetMHCIIpan version 3.1

# Input is in FASTA format

# Peptide length 15

# Threshold for Strong binding peptides (IC50)	50.000 nM
# Threshold for Weak binding peptides (IC50)	500.000 nM

# Threshold for Strong binding peptides (%Rank)	0.5%
# Threshold for Weak binding peptides (%Rank)	2%

# Allele: DRB1_0301
--------------------------------------------------------------------------------------------------------------------------------------------
   Seq          Allele              Peptide    Identity  Pos      Core  Core_Rel 1-log50k(aff)  Affinity(nM)  %Rank Exp_Bind  BindingLevel
--------------------------------------------------------------------------------------------------------------------------------------------
     0         DRB1_0301     VQALLCQFGTVQHVS  CD177_W33S    5    CQFGTVQHV 0.520         0.238       3803.21  65.00   9.999       
     1         DRB1_0301     QALLCQFGTVQHVSK  CD177_W33S    4    CQFGTVQHV 0.475         0.261       2952.53  55.00   9.999       
    